# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [63]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
#%load_ext pycodestyle_magic

In [432]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1-l_s/(l_s+r_s)*np.sum((l_c/l_s)**2,axis=1).reshape(-1,1)-r_s/(r_s+l_s)*np.sum((r_c/r_s)**2,axis=1).reshape(-1,1)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return -l_s/(l_s+r_s)*np.sum(np.nan_to_num(l_c/l_s*np.log2(l_c/l_s)),axis=1).reshape(-1,1)-r_s/(r_s+l_s)*np.sum(np.nan_to_num(r_c/r_s*np.log2(r_c/r_s)),axis=1).reshape(-1,1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1-l_s/(l_s+r_s)*np.max(l_c/l_s,axis=1).reshape(-1,1)-r_s/(r_s+l_s)*np.max(r_c/r_s,axis=1).reshape(-1,1)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return # Ваш код в 1 строчку
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return # Ваш код в 1 строчку

    def __get_feature_ids_N(self, n_feature):
        return # Ваш код
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] < threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода? сортируем матрицу x | y по возрастанию первого столбца,
        # считаем количество различных классов в y
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода? вырезаем из отсортированного столбца классов серединку
        # размера n - 2*min_samples_split
        # находим индексы, на которых происходит смена класса (правые границы классопостоянных участков y)
        cut_size = self.min_samples_split
        splitted_sorted_y = sorted_y[cut_size:len(sorted_y)-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (cut_size + 1)
        
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода? считаем длины классопостоянных участков y (eq_el_count)
        # в результате нескольких нехитрых шагов получаем матрицу class_increments, где
        # каждая строка обозначает порог смены класса, каждый столбик обозначает класс,
        # а на пересечении стоит количество элементов этого класса на данном участке
        # классопостоянства в y
        # в нулевой строке class_increments стоят количества классов в sorted_y до первого порога
        # P.S. из-за min_samples_split там может быть не один класс
        eq_el_count = r_border_ids - np.append([cut_size], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], self.num_class))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:cut_size], minlength=self.num_class)
        
        # Что делает этот блок кода? l_class_count и r_class_count это матрицы, где столбцы
        # отвечают за классы, а строки за пороги (места смены класса). В строках расположены
        # количества классов, которые при отсечении по этому порогу попадут, соответственно,
        # в левый и правый узлы дерева
        # l_sizes и r_sizes это, соответственно, количества сэмплов, попавшие в левый и правый
        # узлы дерева при отсечении по данному порогу (l_sizes и r_sizes это столбики)
        l_class_count = np.cumsum(class_increments, axis=0)   
        r_class_count = np.bincount(sorted_y, minlength=self.num_class) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода? считаем значение функции неопределенности для каждого порога разбиения,
        # находим номер порога, у которого значение наименьшее
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
    
        # Что делает этот блок кода? left_el_id - первый элемент, который попадет в правый лист
        # возвращаем найденный минимум функции неопределенности и значение порога разбиения
        # значение порога разбиения - среднее между значением признака для элемента left_el_id
        # и значением признака для предыдущего элемента, то есть ровно такое значение, чтобы данная
        # выборка разбилась оптимально
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE v
        # self.NON_LEAF_TYPE v

        # self.tree v
        # self.max_depth v
        # self.sufficient_share v
        # self.min_samples_split v

        # self.get_feature_ids
        # self.__find_threshold v
        # self.__div_samples v
        # self.__fit_node v
        
        self.tree[node_id] = [0, 0, 0]
        ncs = np.bincount(y, minlength=self.num_class)
        ncs = ncs.astype('float')
        share = np.max(ncs/float(len(y)))
        if (len(y) <= self.min_samples_split) or (depth == self.max_depth) or (share >= self.sufficient_share):
            self.tree[node_id][0] = self.LEAF_TYPE
            self.tree[node_id][1] = np.argmax(ncs)
            self.tree[node_id][2] = ncs/float(len(y))
        else:
            thresholds = []
            gss = []
            for f_id in range(self.num_class):
                f_gs, f_threshold = self.__find_threshold(x[:, f_id], y)
                thresholds.append(f_threshold)
                gss.append(f_gs)
            np_gss = np.array(gss)
            best_split_id = np.argmin(np_gss)
            best_threshold = thresholds[best_split_id]
            best_feature_id = best_split_id
            
            x_left, x_right, y_left, y_right = self.__div_samples(x, y, best_feature_id, best_threshold)
            while (np_gss[best_split_id] < 1000.) and ((len(y_left) == 0) or (len(y_right) == 0)):
                np_gss[best_split_id] = 1000.
                best_split_id = np.argmin(np_gss)
                best_threshold = thresholds[best_split_id]
                best_feature_id = best_split_id
                x_left, x_right, y_left, y_right = self.__div_samples(x, y, best_feature_id, best_threshold)
            if (len(y_left) == 0) or (len(y_right) == 0):
                self.tree[node_id][0] = self.LEAF_TYPE
                self.tree[node_id][1] = np.argmax(ncs)
                self.tree[node_id][2] = ncs/float(len(y))
            else:
                self.tree[node_id][0] = self.NON_LEAF_TYPE
                self.tree[node_id][1] = best_feature_id
                self.tree[node_id][2] = best_threshold
                self.__fit_node(x_left, y_left, 2*node_id + 1, depth + 1)
                self.__fit_node(x_right, y_right, 2*node_id + 2, depth + 1)
        
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] < threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [469]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, criterion='gini')
clf = DecisionTreeClassifier(min_samples_split=2)

In [470]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [471]:
%time clf.fit(X_train, y_train)

CPU times: user 3.53 ms, sys: 0 ns, total: 3.53 ms
Wall time: 2.16 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [472]:
%time my_clf.fit(X_train, y_train)

CPU times: user 13.1 ms, sys: 0 ns, total: 13.1 ms
Wall time: 11.5 ms


## Проверка качества работы на wine

In [473]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8857142857142858

In [474]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.7583333333333334

## Подготовка данных Speed Dating Data 

In [ ]:
# тут делаете то же самое, что и на семинаре https://github.com/stroykova/spheremailru/blob/master/2018-02/lecture_04_trees/pract-speed-dating-trees-proc.ipynb

## Проверка скорости работы на Speed Dating Data 

In [ ]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

In [ ]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

## Проверка качества работы на Speed Dating Data

In [ ]:
# тут должен быть код типа %time clf.fit(X_train, y_train)

In [ ]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)

## Задание 3

## Задание 4

## Задание 5